<a href="https://colab.research.google.com/github/ywttwy2280/CNPoet_AI/blob/main/CoLab_RAGtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import google.generativeai as genai

GOOGLE_API_KEY = "AIzaSyAhOdVBfjaPWdXtcFywQfp350fwkuQJJI"  # Directly assign API key
genai.configure(api_key=GOOGLE_API_KEY)

In [33]:
from google.colab import files
uploaded = files.upload()

Saving poems.json to poems.json


In [44]:
import json

# Replace with your uploaded file's name
json_file = "poems.json"

with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)


In [39]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert JSON data to Documents
# Use "title" key if it contains the main content
# You might need to adjust the key based on your JSON structure
docs = [Document(page_content=item["title"]) for item in data]

# Store in ChromaDB
vector_db = Chroma.from_documents(docs, embedding_model)

print("✅ Data stored in ChromaDB!")

✅ Data stored in ChromaDB!


In [40]:
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [43]:
# Gemini LLM model
model = genai.GenerativeModel('gemini-2.0-flash')

def rag_query(query):
    # Retrieve relevant context
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])

    # Generate response using Gemini
    response = model.generate_content(f"Context: {context}\n\nQuestion: {query}")

    return response.text

# Example query
query = "翻译成英文：徐志摩《我不知道风是在哪一个方向吹》"
response = rag_query(query)
print("诗歌英文版:", response)

诗歌英文版: Okay, here are a few English translations of Xu Zhimo's "我不知道风是在哪一个方向吹" (Wo bu zhidao feng shi zai na yige fangxiang chui - I don't know which direction the wind is blowing), along with some considerations and different styles:

**Option 1:  A Literal and Simple Approach**

> I don't know which direction the wind is blowing.
> I'm always drifting in dreams.
> I dreamed of sailing on a flower boat,
> Seeking you, seeking you...

**Explanation:** This is a very straightforward translation, aiming for accuracy in meaning.  It preserves the simple, direct language of the original.  It's a good starting point.

**Option 2:  More Poetic and Evocative**

> I know not whence the wind may blow,
> Drifting in dreams, I come and go.
> Dreaming I sail on a flowered barge,
> Seeking you, seeking you, in the large... (sea/world/vastness)

**Explanation:** This version uses slightly more archaic and poetic language ("whence," "come and go," "flowered barge") to give a more literary feel, close